## Bullet Bonds


A bullet bond is a type of fixed-income security with a very straightforward repayment structure.

### Key Features
1. **No Amortization**  
   - The issuer does not repay any portion of the principal during the life of the bond.  
   - Only interest (coupon) payments are made periodically (e.g., semi-annual or annual).

2. **Principal Repaid at Maturity**  
   - The entire face value (principal) is repaid in one "bullet" payment at the end of the bond’s term.  
   - This makes it different from amortizing bonds (like many mortgages), where principal is gradually repaid.

3. **Predictable Cash Flow**  
   - Investors receive regular coupon payments and then a large lump sum at maturity.  
   - This makes bullet bonds relatively simple to understand and model.

### Advantages
- **For issuers**: Lower cash outflow during the life of the bond since they don’t repay principal until maturity.  
- **For investors**: Predictable stream of income plus a known lump-sum repayment date.

### Risks
- **Credit risk**: Since principal repayment is concentrated at the end, investors bear the risk that the issuer might default before maturity.  
- **Refinancing risk for issuers**: They must have sufficient liquidity or refinancing ability to repay the lump sum when due.  

### Common Examples
- Many **corporate bonds** and **sovereign bonds** are structured as bullet bonds.  
- Eurobonds and U.S. Treasury notes/bonds are classic examples.



In [33]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

from IPython.display import Markdown

bond = FixedRateBullet("2020-01-01", "2025-01-01", 5, 4, notional=100)


def display_bond_cash_flows(cash_flow_dict):
    table = "## Bond Cash Flows Table\n\n"
    table += "| Date       | Cash Flow Value |\n"
    table += "|------------|-----------------|\n"
    for time, value in sorted(cash_flow_dict.items(), key=lambda x: x[0]):
        table += f"| {time.strftime('%Y-%m-%d')} | {value:.2f}            |\n"
    display(Markdown(table))


display_bond_cash_flows(bond.payment_flow)

## Bond Cash Flows Table

| Date       | Cash Flow Value |
|------------|-----------------|
| 2020-04-01 | 1.25            |
| 2020-07-01 | 1.25            |
| 2020-10-01 | 1.25            |
| 2021-01-01 | 1.25            |
| 2021-04-01 | 1.25            |
| 2021-07-01 | 1.25            |
| 2021-10-01 | 1.25            |
| 2022-01-01 | 1.25            |
| 2022-04-01 | 1.25            |
| 2022-07-01 | 1.25            |
| 2022-10-01 | 1.25            |
| 2023-01-01 | 1.25            |
| 2023-04-01 | 1.25            |
| 2023-07-01 | 1.25            |
| 2023-10-01 | 1.25            |
| 2024-01-01 | 1.25            |
| 2024-04-01 | 1.25            |
| 2024-07-01 | 1.25            |
| 2024-10-01 | 1.25            |
| 2025-01-01 | 101.25            |


## Bond Day-Counting Conventions

These rules decide how to count days for calculating bond interest:

- **30/360**: Counts each month as 30 days, year as 360 days. Simple but less accurate. Used in some corporate bonds.
- **30E/360**: Like 30/360, but if a date is the 31st, it’s counted as the 30th. Used in Eurobonds.
- **Actual/Actual**: Counts real days in the period and year (365 or 366). Most accurate. Used in US Treasuries (ISDA) or international bonds (ICMA).
- **Actual/360**: Counts real days in the period, but uses a 360-day year. Common in money markets like loans.
- **Actual/365**: Counts real days in the period, uses a 365-day year. Used in some bond markets.
- **30/365**: Counts each month as 30 days, year as 365 days. Rare, used in specific financial contracts.
- **Actual/Actual-Bond**: Same as Actual/Actual (ICMA), counts real days in period and coupon period (e.g., half-year). Used in bond markets.

In [34]:
bond_1 = FixedRateBullet(
    "2020-01-01", "2025-01-01", 5, 2, notional=100, day_count_convention="30/360"
)
bond_2 = FixedRateBullet(
    "2020-01-01", "2025-01-01", 5, 2, notional=100, day_count_convention="30e/360"
)
# bond_3 = FixedRateBullet('2020-01-01', '2025-01-01', 5, 1, notional=100, day_count_convention = 'actual/actual')
bond_4 = FixedRateBullet(
    "2020-01-01", "2025-01-01", 5, 2, notional=100, day_count_convention="actual/360"
)
bond_5 = FixedRateBullet(
    "2020-01-01", "2025-01-01", 5, 2, notional=100, day_count_convention="actual/365"
)
bond_6 = FixedRateBullet(
    "2020-01-01", "2025-01-01", 5, 2, notional=100, day_count_convention="30/365"
)
bond_7 = FixedRateBullet(
    "2020-01-01",
    "2025-01-01",
    5,
    2,
    notional=100,
    day_count_convention="actual/actual-Bond",
)


# --- Pretty day-count label helper ---
def dcc_label(dcc_obj):
    """
    Return a clean, human-readable label for common day-count objects.
    Works if the bond stores: a) a string like '30/360', or
    b) an instance like DayCount30360, DayCountActualActualBond, etc.
    """
    # If already a clean string, just use it
    if isinstance(dcc_obj, str):
        return dcc_obj

    # If it exposes a 'name' or similar
    for attr in ("name", "label", "code"):
        if hasattr(dcc_obj, attr):
            val = getattr(dcc_obj, attr)
            if isinstance(val, str) and val.strip():
                return val

    # Fall back to class name mapping
    cls = dcc_obj.__class__.__name__

    # Common mappings for pyfian-style names
    mapping = {
        "DayCount30360": "30/360",
        "DayCount30E360": "30E/360",
        "DayCountActual360": "ACT/360",
        "DayCountActual365": "ACT/365",
        "DayCount30365": "30/365",
        "DayCountActualActualBond": "ACT/ACT (Bond)",
        "DayCountActualActual": "ACT/ACT",
    }

    if cls in mapping:
        return mapping[cls]

    # Generic fallback: split CamelCase and add spaces
    import re

    spaced = re.sub(r"(?<!^)(?=[A-Z])", " ", cls)
    return spaced


# --- Display function in your style ---
def display_bond_accrued_interest(bond_dict, settlement_date="2023-03-01"):
    table = f"## Accrued Interest Table (Settlement Date: {settlement_date})\n\n"
    table += "| Bond   | Day Count Convention | Accrued Interest |\n"
    table += "|--------|----------------------|------------------|\n"

    for name, bond in bond_dict.items():
        # Try to fetch the day-count stored on the bond
        dcc = getattr(bond, "day_count_convention", None)
        label = dcc_label(dcc) if dcc is not None else "(unknown)"

        # Compute accrued interest (handle alternate param name)
        try:
            ai = bond.accrued_interest(settlement_date=settlement_date)
        except TypeError:
            ai = bond.accrued_interest(settlement=settlement_date)

        # Format numeric nicely
        ai_str = f"{ai:.6f}" if isinstance(ai, (int, float)) else str(ai)
        table += f"| {name} | {label} | {ai_str} |\n"

    display(Markdown(table))


# --- Usage with your bonds ---
bonds = {
    "bond_1": bond_1,  # 30/360
    "bond_2": bond_2,  # 30E/360
    "bond_4": bond_4,  # ACT/360
    "bond_5": bond_5,  # ACT/365
    "bond_6": bond_6,  # 30/365
    "bond_7": bond_7,  # ACT/ACT (Bond)
}

display_bond_accrued_interest(bonds, settlement_date="2023-03-01")

## Accrued Interest Table (Settlement Date: 2023-03-01)

| Bond   | Day Count Convention | Accrued Interest |
|--------|----------------------|------------------|
| bond_1 | 30/360 | 0.416667 |
| bond_2 | 30E/360 | 0.416667 |
| bond_4 | actual/360 | 0.409722 |
| bond_5 | actual/365 | 0.404110 |
| bond_6 | 30/365 | 0.410959 |
| bond_7 | actual/actual-Bond | 1.629834 |


### Calculating Bond Price from Discount Curve

The price of a bond is the present value of its future cash flows, which consist of periodic coupon payments and the face value paid at maturity. A discount curve provides the discount factors needed to compute these present values, accounting for the time value of money and interest rate risk. Each cash flow is multiplied by the discount factor corresponding to its payment time, and the sum of these discounted cash flows gives the bond's price.

Consider a 5-year bond with:
- **Face Value**: $100
- **Annual Coupon Rate**: 5% (paid semi-annually, so $2.50 every 6 months)
- **Maturity**: 5 years

##### 1) Identify Cash Flows
- **Coupon Payments**: $2.50 at `t = 0.5, 1.0, ..., 4.5` years (semi-annual).
- **Final Payment**: $102.50 at `t = 5.0` years (face value + final coupon).

##### 2) Apply Discount Curve
Using the `DummyCurve` discount factor, defined as `discount_t(t) = 1 / (1 + 0.05 * t)`, each cash flow is discounted:
- At `t = 0.5`: $2.50 / (1 + 0.05 * 0.5) ≈ $2.439$
- At `t = 5.0`: $102.50 / (1 + 0.05 * 5.0) = $102.50 / 1.25 = $82.0$

##### 3) Calculate Bond Price
Sum the discounted cash flows:
- $2.439 + $2.381 + ... + $82.0 ≈ $102.066$

This total represents the bond's price, reflecting the present value of all future payments adjusted by the discount curve.

In [ ]:
# fix this later

TypeError: unsupported operand type(s) for -: 'Timestamp' and 'datetime.date'



## Duration

Duration measures a bond's price sensitivity to changes in interest rates and represents the weighted average time to receive the bond's cash flows. Mathematically, duration approximates the percentage change in a bond's price for a small change in yield, expressed as:

$$D = -\frac{1}{P} \cdot \frac{dP}{dr}$$

where $P$ is the bond's price and $r$ is the yield (interest rate). The negative sign reflects the inverse relationship between bond prices and yields.

### Macaulay Duration

Macaulay duration is the weighted average time until a bond's cash flows are received, expressed in years. It is calculated as:

$$\frac{\sum_{t=1}^n \frac{t \cdot C_t}{(1 + y)^t} + \frac{n \cdot FV}{(1 + y)^n}}{P}$$

where:
- $C_t$: Cash flow (coupon payment) at time $t$
- $FV$: Face value of the bond
- $y$: Yield to maturity (per period)
- $n$: Number of periods until maturity
- $P$: Present value (current price) of the bond

For a zero-coupon bond, since there are no coupon payments ($C_t = 0$), the Macaulay duration simplifies to the time to maturity, $T - t$, where $T$ is the maturity date and $t$ is the current date.

### Modified Duration

Modified duration directly measures the percentage change in a bond's price for a 1% change in yield. It is derived from the Macaulay duration as:

$$D^{\text{Mod}} = \frac{D^{\text{MC}}}{1 + \frac{y}{k}}$$

where:
- $y$: Annual yield to maturity
- $k$: Number of compounding periods per year (e.g., $k = 2$ for semi-annual, $k = 1$ for annual)

For a zero-coupon bond with annual compounding ($k = 1$), the modified duration is:

$$D^{\text{Mod}} = \frac{T - t}{1 + y}$$

### Example of Duration

For a zero-coupon bond, the percentage change in the bond’s price for a 1% (or 0.01) change in yield is approximately:

$$\frac{\Delta P}{P} \approx -D^{\text{Mod}} \times \Delta y$$

**Example**: Consider a zero-coupon bond with 5 years to maturity ($T - t = 5$) and a yield to maturity of 4% ($y = 0.04$) with annual compounding. The modified duration is:

$$D^{\text{Mod}} = \frac{5}{1 + 0.04} = \frac{5}{1.04} \approx 4.8077$$

For a 1% increase in yield ($\Delta y = 0.01$):

$$\frac{\Delta P}{P} \approx -4.8077 \times 0.01 \approx -0.048077 \text{ or } -4.81\%$$

Thus, the bond’s price decreases by approximately 4.81% if the yield increases by 1%.

## Convexity

Convexity measures the sensitivity of a bond’s duration to changes in interest rates and captures the non-linear relationship between bond prices and yields. It is the second derivative of the bond’s price with respect to the yield, scaled by the price:

$$C = \frac{1}{P} \cdot \frac{d^2P}{dr^2}$$

For a zero-coupon bond with annual compounding, the convexity is:

$$C = \frac{(T - t)(T - t + 1)}{(1 + y)^2}$$

**Example**: For the same zero-coupon bond ($T - t = 5$, $y = 0.04$):

$$C = \frac{5 \cdot 6}{(1 + 0.04)^2} = \frac{30}{1.0816} \approx 27.73$$

Convexity improves the accuracy of price change estimates for larger yield changes by accounting for the curvature of the price-yield relationship.

In [ ]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet




## Effective Duration 

Effective Duration finds the duration for bonds that have options embedded into them. For example a callable bond is an example of a bonds that has an option embedded into it. 

### Spread Effective Duration

The spread effective duration measures how much a bond price changes with respect to a change in the bond's credit spread. 